In [ ]:
import azureml.core
from azureml.core import Workspace, Environment, Experiment, Datastore, Dataset, ScriptRunConfig
from azureml.core.compute import ComputeTarget, AmlCompute, DatabricksCompute
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import RunConfiguration
from azureml.exceptions import ComputeTargetException
from azureml.pipeline.steps import HyperDriveStep, HyperDriveStepRun, PythonScriptStep, DatabricksStep
from azureml.pipeline.core import Pipeline, PipelineData, TrainingOutput
from azureml.train.hyperdrive import RandomParameterSampling, BanditPolicy, HyperDriveConfig, PrimaryMetricGoal
from azureml.train.hyperdrive import choice, loguniform
from azureml.data.data_reference import DataReference
from azureml.pipeline.core import PipelineParameter
from azureml.pipeline.core import Pipeline, StepSequence

In [ ]:
#Set up the compute of the ML pipeline, which should be compute cluster
ws = Workspace.from_config()
cluster_name = "samplename"
 
compute_target = ComputeTarget(workspace=ws, name=cluster_name)
print(compute_target.get_status().serialize())

In [ ]:
#Set up the datastore that you will use for input data
def_blob_store = ws.get_default_datastore()

In [ ]:
# Choose an appropriate Azure cloud ML environment, or you could also customized your own customized environment based on personal needs
env = Environment.get(ws, name="AzureML-sklearn-1.0-ubuntu20.04-py38-cpu")

In [ ]:
# Setup the RunConfiguration object, it is necessary for submitting any runs in an experiment
rcfg = RunConfiguration()
rcfg.environment = env

Build customized Pipeline Steps:
a. Now we are creating some pipeline steps in sequence for my demo pipeline. There are many ways to create each pipeline step, and you can design as many steps as you need with any sequences for your ML pipeline. For this demo pipeline, I will only create 2 pipeline steps. by using the PythonScriptStep. The first pipeline step is called "Data_Clean", and the subsequent step is called "Model_Train" 
b. Build the first pipeline step: Data_Clean. The purpose of this step is for data cleansing (pipeline parameter):
Details of PIpelineparameter Class:https://learn.microsoft.com/en-us/python/api/azureml-pipeline-core/azureml.pipeline.core.graph.pipelineparameter?view=azure-ml-py 

Make sure that the datapath URI actually exits in the datastore which you have setup connection with the data container, and make sure is is the type of Datasotre URI for the

In [ ]:
from azureml.core.datastore import Datastore
from azureml.data.datapath import DataPath, DataPathComputeBinding
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import PipelineParameter
 
datapath = 'Datastore URI1'
datapath2 = 'Datastore URI2'
data_path_pipeline_param = PipelineParameter(name="input_data_train", default_value=datapath)
data_path_pipeline_param2 = PipelineParameter(name="input_data_predict", default_value=datapath2)

In [ ]:
# Set up output data destinations. For this demo, I setup 2 output data configuration. One is for the cleaned training data , and the other one is for the cleaned input data that we need to make prediction later
from azureml.data import OutputFileDatasetConfig
 
output_train = OutputFileDatasetConfig(name="demo1_processed_train",
                                 destination=(def_blob_store, "")).read_delimited_files().register_on_complete('demo1_processed_train')
 
output_inputdata = OutputFileDatasetConfig(name="demo1_processed_inputdata",
                                 destination=(def_blob_store, "")).read_delimited_files().register_on_complete('demo1_processed_inputdata')

In [ ]:
# Define the pipeline step1 to trigger the 'data_cleaning3.py' script. For the 'data_cleaning.py' script: data_cleaning.py
data_clean = PythonScriptStep(script_name='data_cleaning.py',
                                       source_directory=script_folder,
                                       name="Data_Clean",
                                       compute_target=compute_target,
                                       arguments=['--input_data_path_train', data_path_pipeline_param,
                                                  '--input_data_path_predict', data_path_pipeline_param2,
                                                  '--processed_data_train', output_train,
                                                  '--processed_data_inputdata', output_inputdata],
                                    #    inputs=[datapath_input],
                                       allow_reuse=True,
                                       runconfig=rcfg)

In [ ]:
# Build the second pipeline step: model_train. The purpose of this step to train the model with registering the model to AML portal, and produce the prediction output to the target azure data container. 
model_train = PythonScriptStep(script_name='model_train.py',
                                      source_directory=script_folder,
                                      name="Model_Train",
                                      compute_target=compute_target,
                                      arguments=['--cleaned_data_train', output_train.as_input('cleaned_data_train'),
                                                 '--cleaned_data_predict', output_inputdata.as_input('cleaned_data_predict')],
                                   #    inputs=[datapath_input],
                                      allow_reuse=True,
                                      runconfig=rcfg)

In [ ]:
pipeline = Pipeline(workspace=ws, steps=[Input_Data, Data_Clean, Model_train, Register_Model, Data_output])
pipeline_run = exp.submit(pipeline)

In [ ]:
pipeline_run = exp.submit(pipeline)

Publish pipeline to an endpoint

In [ ]:
from azureml.pipeline.core import PipelineEndpoint
 
pipeline_name = "Demo_Endpoint"
 
if [x for x in PipelineEndpoint.list(ws) if x.name == pipeline_name]:
    pipeline_endpoint = PipelineEndpoint.get(ws, name=pipeline_name)
    pipeline_endpoint.add_default(published_pipeline)
else:
    pipeline_endpoint = PipelineEndpoint.publish(workspace=ws, name=pipeline_name,
                                             pipeline=published_pipeline, description="....")

Deployment Step 2: Setup a scheduler on Synapse for the ML pipeline endpoint
Linked the ML service on Synapse, and trigger it with a scheduler 